In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from collections import defaultdict

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
WS_REAL = defaultdict(list)
valid_final_season = {}

FEAT = ['Age','WS']

for YR in range(1980,2020):
    table = np.load('tables_{}.pkl'.format(YR),allow_pickle=True)
    for team in table:
        stats = table[team]['advanced'][FEAT]
        for item in stats.itertuples():
            try:
                v = [0.0 if _=='' else _ for _ in item[1:]]
                #hgt = table[team]['roster'].loc[item[0]]['Ht']
                #hgt = [int(_) for _ in hgt.split('-')]
                #hgt = 12*hgt[0] + hgt[1]
                #v.append(hgt)
                WS_REAL[item[0]].append(np.array(v).astype(np.float))
            except:
                print(item)

In [ ]:
d = []
for k in WS_REAL:
    t = np.array(WS_REAL[k])
    tm = t[:,1]#*t[:,3]
    tm = tm.astype(np.float)
    d.append(np.vstack([[k for _ in range(tm.shape[0])],tm,t[:,1]]).T)

In [ ]:
table = np.load('tables_{}.pkl'.format(2019),allow_pickle=True)
for team in table:
    stats = table[team]['advanced']
    for item in stats.itertuples():
        valid_final_season[item[0]] = 1


In [ ]:
X = []
y = []
for name in WS_REAL:
    stats = WS_REAL[name]
    yrs = len(stats)
    X += stats
    for i in range(yrs-1):
        y.append(0.0)
    y.append(float(name not in valid_final_season))

In [ ]:
from sklearn.linear_model import LogisticRegression, ElasticNet, Ridge,RidgeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

In [ ]:
Xo = np.array(X)
yo = np.array(y)


In [ ]:
X = Xo.copy()
y = yo.copy()
X[:,0] -= 18
X[:,0] *= X[:,0]
#X[:,-1] /= X[:,-1].max()
#X = np.hstack([X,(X[:,0] * X[:,1]).reshape((-1,1))])

In [ ]:
fexp = PolynomialFeatures(2,interaction_only=True)
scal = StandardScaler()
pX = X#fexp.fit_transform(X)#fexp.fit_transform(X)
#pX[:,0] = pX[:,0]**2

clf = LogisticRegression(C=1e6,solver='liblinear')#,class_weight='balanced')
clf.fit(pX,y)
clf.score(pX,y),(1-y.sum()/y.shape[0])

In [ ]:
_ = plt.hist(clf.predict_proba(pX)[:,1],50,density=True)

In [ ]:
FEAT,clf.coef_,clf.intercept_

In [ ]:
import statsmodels.api as sm

In [ ]:
FEAT

In [ ]:
X2 = sm.add_constant(pX)
df_tmp = pd.DataFrame(X2,columns=['1'] + FEAT )
df_tmp.columns = ['(Age-18)^2' if _ == 'Age' else _ for _ in df_tmp.columns]
logit_mod = sm.Logit(y,df_tmp)#fexp.get_feature_names(FEAT)))

logit_res = logit_mod.fit(disp=0)
print()
np.mean((logit_res.predict() >0.5)== y.astype(np.bool))
logit_res.summary()

In [ ]:
xp = np.linspace(18,50)
p = lambda x: 1.0/(1.0+np.exp(-x))
#p = lambda x: np.log(np.exp(x)+1)
for WS in [-1,0,1,3,5,7]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(logit_res.params[0] + logit_res.params[1]*(xp-18)**2 + logit_res.params[2]*WS   ),label='WS {}'.format(WS))
plt.xlim(18,55)
plt.legend()
plt.xlabel('Age')

plt.ylabel('Prob of Retiring')
plt.title('NBA (Age-18)^2 + WS Model')
plt.ylim(0,1)
plt.xlim(18,50)
plt.tight_layout()
plt.grid(True)

plt.savefig('retire.png',facecolor='w',edgecolor='w')

In [ ]:
xp = np.linspace(18,60)
p = lambda x: 1.0/(1.0+np.exp(-x))
#p = lambda x: np.log(np.exp(x)+1)

for WS in [-1,0,1,3,5,7]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,100*p(clf.intercept_ + clf.coef_[0,0]*(xp-18)**2 + clf.coef_[0,1]*WS   ),label='WS {}'.format(WS))
plt.xlim(18,55)
plt.legend()
plt.xlabel('Age')

plt.ylabel('Prob of Retiring')
plt.title('New BBGM Model')
plt.ylim(0,100)
plt.xlim(35,60)

plt.tight_layout()
plt.grid(True)

plt.savefig('retire.png',facecolor='w',edgecolor='w')

In [ ]:
pX

In [ ]:
df2 = pd.read_csv('beta_stats.csv')#big_stat


In [ ]:
from matplotlib.colors import LogNorm
plt.style.use('fivethirtyeight')
dft = df2[df2.MP*df2.G > 300]
_ = plt.hexbin(dft.Ovr,dft['WS'],gridsize=40,norm=LogNorm())
#plt.ylim(-5,40)
#plt.xlim(30,85)
plt.xlabel('Ovr')
plt.ylabel('WS')
xp = np.linspace(30,90)
#plt.plot(xp,(1/64)*np.maximum(xp-40,0)**2,c='r')
plt.title('bbgm ovr -> WS')
plt.savefig('wsovr3.png',edgecolor='w',facecolor='w')

In [ ]:
WS_FAKE = defaultdict(list)
valid_final_season2 = {}

FEAT2 = ['Name','Age','WS','TS%','3PAr','Pot']

for item in df2[FEAT2].itertuples():
    try:
        v = [0.0 if _=='' else _ for _ in item[2:]]
        #hgt = table[team]['roster'].loc[item[0]]['Ht']
        #hgt = [int(_) for _ in hgt.split('-')]
        #hgt = 12*hgt[0] + hgt[1]
        #v.append(hgt)
        WS_FAKE[item[1]].append(np.array(v).astype(np.float))
    except:
        raise#print(item)

In [ ]:
X2 = []
y2 = []
for name in WS_FAKE:
    stats = WS_FAKE[name]
    yrs = len(stats)
    X2 += stats
    for i in range(yrs-1):
        y2.append(0.0)
    y2.append(1.0)
    if len(y2) != len(X2):
        print('omg',len(y2),len(X2),name)

In [ ]:
X2 = np.array(X2)
y2 = np.array(y2)


In [ ]:

clf2 = LogisticRegression(C=1e6,solver='liblinear')#,class_weight='balanced')
clf2.fit(X2,y2)
clf2.score(X2,y2),(1-y2.sum()/y2.shape[0])

In [ ]:
_ = plt.hist(clf2.predict_proba(X2)[:,1],50,density=True)

In [ ]:
FEAT2,clf2.coef_,clf2.intercept_

In [ ]:
X3 = sm.add_constant(X2)
X3 = pd.DataFrame(X3,columns=['1'] + FEAT2[1:])
logit_mod = sm.Logit(y2,X3)
logit_res = logit_mod.fit(disp=0)
print(logit_res.summary())
np.mean((logit_res.predict() >0.5)== y2.astype(np.bool))

In [ ]:
xp = np.linspace(20,40)
p = lambda x: 1.0/(1.0+np.exp(-x))
for WS in [-1,0,1,3,5,7]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(clf2.intercept_ + clf2.coef_[0,0]*xp + clf2.coef_[0,1]*WS+ clf2.coef_[0,2]*30+ clf2.coef_[0,3]*5 + clf2.coef_[0,4]*50  ),label='WS {}'.format(WS))
plt.xlim(20,46)
plt.legend()
plt.title('BBGM, TS%:30, 3PAr:5')
plt.xlabel('Age')
plt.ylabel('P(Retire | Age, WS)')
plt.tight_layout()
plt.savefig('retireFAKE.png',facecolor='w',edgecolor='w')

In [ ]:
df2['OvrSm'] = (np.maximum(0,df2['Ovr']-37))**2

In [ ]:
df3 = df2[df2.Age > 22]

In [ ]:
clf_ws = sm.OLS(df3.WS,df3['OvrSm']).fit()
from scipy.stats import pearsonr
print(pearsonr(clf_ws.predict(),df3.WS)[0])
clf_ws.summary()


In [ ]:
plt.scatter(df3.WS,clf_ws.predict(),alpha=0.5,s=8)

In [ ]:
clf3 = ElasticNet(alpha=0,fit_intercept=False)
#df3 = df2[df2.Age > 22]
clf3.fit(np.array(df3.OvrSm).reshape((-1,1)),df3.WS)
clf3.score(np.array(df3.OvrSm).reshape((-1,1)),df3.WS)

In [ ]:
xp = np.linspace(18,60)
p = lambda x: 1.0/(1.0+np.exp(-x))
#p = lambda x: np.log(np.exp(x)+1)

for OVR in [50,55,60,65]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,100*p(clf.intercept_ + clf.coef_[0,0]*(xp-18)**2 + clf.coef_[0,1]* clf3.coef_[0]*(OVR-37)**2   ),label='OVR {}'.format(OVR))
plt.xlim(18,55)
plt.legend()
plt.xlabel('Age')

plt.ylabel('Prob of Retiring')
plt.title('New BBGM Retirement')
plt.ylim(0,100)
plt.xlim(35,60)

plt.tight_layout()
plt.grid(True)

plt.savefig('retire3.png',facecolor='w',edgecolor='w')

In [ ]:
clf3.coef_,clf3.intercept_,clf3.coef_[0]

In [ ]:
clf3.coef_[0]*50+clf3.intercept_

In [ ]:
xp = np.linspace(20,60)
p = lambda x: 1.0/(1.0+np.exp(-x))
for POT in [40,50,60,70]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*(xp-18)**2 + clf.coef_[0,1]* (clf3.coef_[0]*((POT-37)**2)) ),label='Pot {}'.format(POT))
plt.xlim(20,55)
plt.ylim(0,1)
plt.legend()
plt.xlabel('Age')

plt.ylabel('P(Retire | Age, Pot)')
plt.title('Overall model')

plt.tight_layout()

plt.savefig('retire2.png',facecolor='w',edgecolor='w')

In [ ]:
clf.intercept_ ,1/clf.coef_[0,0],clf.coef_[0,1]

In [ ]:
df2['retire'] = clf.intercept_ + clf.coef_[0,0]*(np.maximum(df2.Age,18)-18)**2  + clf.coef_[0,1]*np.maximum(df2.WS,clf3.coef_[0]*((np.maximum(37,df2.Pot)-37)**2))

df2['retire'] = 1/(1+np.exp(-df2.retire))

In [ ]:
1/clf3.coef_,clf3.intercept_

In [ ]:
plt.hist(df2.retire,20)

In [ ]:
retired = df2[df2.retire >0.5][['WS','Age','Pot','retire']]
retired

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(10,6))
for i,col in enumerate(retired.columns):
    plt.subplot(2,2,1+i)
    plt.hist(df2[df2.retire < 0.5][col],20,density=True,alpha=0.8,label='not retired')
    plt.hist(df2[df2.retire > 0.5][col],8,density=True,alpha=0.8,label='retired')
    plt.title('Retirement ' + col + '\n (weight: Balanced)')
    if i == 0:
        plt.xlim(-6,10)
    plt.legend()
plt.tight_layout()

In [ ]:
plt.hexbin(df2.Age,df2.retire,gridsize=17)
plt.xlabel('Age')
plt.ylabel('P(Retire)')
plt.title('Balanced')
plt.colorbar()

In [ ]:
dft = df2[(df2.Ovr < 62) & (df2.WS > 16)]
dft

In [ ]:
pd.set_option("display.precision", 2)
df2[(df2.Season == 2187) & (df2.Team == 'MIA')].sort_values('MP',0,False)[['Name','Pos','Age','Ovr','Salary','GS','MP','WS','PTS','TRB','AST','STL','Blk','TOV']]

In [ ]:
from tabulate import tabulate

In [ ]:
print(tabulate(df2[(df2.Season == 2187) & (df2.Team == 'MIA')].sort_values('MP',0,False)[['Name','Pos','Age','Salary','GS','MP','WS','PTS','TRB','AST','STL','Blk','TOV']]))